In [55]:
import torch
torch.__version__

'1.0.0.dev20181018'

In [56]:
bs = 8
nc = 5

In [57]:
y_pred = torch.randn(bs, nc)
y_pred = y_pred.sigmoid()
y_pred.shape

torch.Size([8, 5])

In [58]:
y_pred

tensor([[0.4296, 0.4082, 0.3289, 0.2174, 0.5681],
        [0.9369, 0.4079, 0.4381, 0.5847, 0.4054],
        [0.5881, 0.5179, 0.1935, 0.7214, 0.1461],
        [0.3007, 0.3951, 0.1621, 0.2608, 0.6127],
        [0.2047, 0.3238, 0.7656, 0.7414, 0.4226],
        [0.1461, 0.8969, 0.7945, 0.1854, 0.7581],
        [0.6351, 0.6160, 0.7741, 0.4442, 0.7180],
        [0.4873, 0.3132, 0.0923, 0.6354, 0.6763]])

In [59]:
y_true = torch.randint(low=0, high=2, size=(bs, nc))
y_true

tensor([[1, 1, 0, 0, 1],
        [1, 1, 1, 0, 0],
        [1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1],
        [0, 1, 1, 1, 1],
        [1, 1, 0, 0, 1],
        [1, 1, 0, 0, 0]])

In [60]:
y_rank = y_pred.argsort(descending=True)
y_rank

tensor([[4, 0, 1, 2, 3],
        [0, 3, 2, 1, 4],
        [3, 0, 1, 2, 4],
        [4, 1, 0, 3, 2],
        [2, 3, 4, 1, 0],
        [1, 2, 4, 3, 0],
        [2, 4, 0, 1, 3],
        [4, 3, 0, 1, 2]])

In [61]:
k = 10
nc = y_true.shape[1]
assert 0<=k<=nc , f"k ({k}) needs to be less than or equal to the number of classes ({nc})"

AssertionError: k (10) needs to be less than or equal to the number of classes (5)

In [63]:
k = 3
nc = y_true.shape[1]
assert 0<=k<=nc , f"k ({k}) needs to be less than or equal to the number of classes ({nc})"

In [64]:
y_true

tensor([[1, 1, 0, 0, 1],
        [1, 1, 1, 0, 0],
        [1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1],
        [0, 1, 1, 1, 1],
        [1, 1, 0, 0, 1],
        [1, 1, 0, 0, 0]])

In [65]:
y_rank

tensor([[4, 0, 1, 2, 3],
        [0, 3, 2, 1, 4],
        [3, 0, 1, 2, 4],
        [4, 1, 0, 3, 2],
        [2, 3, 4, 1, 0],
        [1, 2, 4, 3, 0],
        [2, 4, 0, 1, 3],
        [4, 3, 0, 1, 2]])

In [66]:
y_reorder = torch.stack([y_true_i[y_rank[i]] for i,y_true_i in enumerate(y_true)])
y_reorder

tensor([[1, 1, 1, 0, 0],
        [1, 0, 1, 1, 0],
        [1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0],
        [1, 1, 1, 1, 0],
        [0, 1, 1, 1, 0],
        [0, 0, 1, 1, 0]])

In [67]:
accuracy_at_k = y_reorder[:, :k].float().mean()

In [68]:
from torch import Tensor
def accuracy_at_k(y_pred:Tensor, y_true:Tensor, k:int, sigmoid:bool=True):
    "Compute accuracy of top `k` predictions when `y_pred` and `y_true` are the same size."
    nc = y_true.shape[1]
    assert 0<=k<=nc , f"k ({k}) needs to be less than or equal to the number of classes ({nc})"    
    if sigmoid: y_pred = y_pred.sigmoid()
    y_rank = y_pred.argsort(descending=True)
    y_reorder_topk = torch.stack([y_true_i[y_rank[i]][:k] for i,y_true_i in enumerate(y_true)])
    return y_reorder_topk.float().mean()

In [69]:
accuracy_at_k(y_pred, y_true, k)

tensor(0.8333)

In [70]:
from functools import partial

In [71]:
accuracy_at_3 = partial(accuracy_at_k, k=3)
accuracy_at_3(y_pred, y_true)

tensor(0.8333)

In [72]:
def accuracy_at_3(y_pred, y_true):
    return accuracy_at_k(y_pred, y_true, k=3)
accuracy_at_3(y_pred, y_true)

tensor(0.8333)